In [1]:
import pandas as pd 
import numpy as np 
from optimisation_utiles import * 
from grm_utiles import * 
import matplotlib.pyplot as plt
from GRM_monte_carlo_utiles import * 

In [2]:
df_return = pd.read_excel('GRM_returns.xlsx')
df_return['Date'] = pd.to_datetime(df_return['Date'])
df_return = df_return.set_index('Date')
df_MXWO = pd.read_excel('MSCI_world.xlsx')
df_MXWO['Date'] = pd.to_datetime(df_MXWO['Date'])
df_MXWO = df_MXWO.set_index('Date')
df_return = df_return[df_return.index>'2007-09-01']
df_MXWO = df_MXWO[df_MXWO.index>'2007-09-01']

In [3]:
df_MXWO_return = df_MXWO.pct_change(1).dropna(0).resample('M').apply(lambda x: (1+x).prod() - 1)
df_monthly = df_return.resample('M').apply(lambda x: (1+x).prod() - 1)

In [17]:
num_months = len(df_MXWO_return)
df_MXWO_topdd = df_MXWO_return.sort_values('MXWO Index')[:int(num_months/20)]
worst_MXWO_months = df_MXWO_topdd.index
df_MXWO_topdd

,MXWO Index
Date,
2008-10-31,-0.190445
2020-03-31,-0.134691
2008-09-30,-0.120777
2009-02-28,-0.104872
2010-05-31,-0.099086
2012-05-31,-0.089916
2011-09-30,-0.088473
2009-01-31,-0.088456


In [18]:
sr_MXWO_dd_threshold = ((-1) * df_MXWO_topdd * 0.25).squeeze().sort_index()
sr_MXWO_dd_threshold

Date
2008-09-30    0.030194
2008-10-31    0.047611
2009-01-31    0.022114
2009-02-28    0.026218
2010-05-31    0.024771
2011-09-30    0.022118
2012-05-31    0.022479
2020-03-31    0.033673
Name: MXWO Index, dtype: float64

In [66]:
gen_random_port_num = 200000
rand_wts = np.random.rand(gen_random_port_num, len(df_monthly.columns))
wts = rand_wts / rand_wts.sum(axis=1).reshape(gen_random_port_num,1).repeat(len(df_monthly.columns), axis=1)
first_risk = wts[:,:11].sum(axis=1) # first_risk
defensive_factors = wts[:,11:20].sum(axis=1) # defensive_factors
trend = wts[:,20:26].sum(axis=1) # trend 
tail_risk = wts[:,26:30].sum(axis=1) # tail_risk

In [67]:
# factor groups weights 
first_risk_bool = (first_risk > 0.15) & (first_risk < 0.4)
defensive_factors_bool = (defensive_factors > 0.3) & (defensive_factors < 0.5)
trend_bool = (trend > 0.1) & (trend < 0.25)
tail_risk_bool = (tail_risk > 0.1) & (tail_risk < 0.24)
wts_filter = wts[first_risk_bool & defensive_factors_bool & trend_bool & tail_risk_bool]

In [68]:
print(len(wts_filter))

65229


In [60]:
# individual weights 

In [69]:
%%time
good_weights_list = []
for i in range(len(wts_filter)):
    if check_meeting_dd_requirements(df_monthly, wts_filter[i], 0.18, df_MXWO_topdd):
        print('find!')
        good_weights_list.append(wts_filter[i])


find!
find!
Wall time: 11min 5s


In [70]:
good_weights_list

[array([0.04572257, 0.02003531, 0.00145188, 0.03452972, 0.00013168,
        0.01089242, 0.01242946, 0.07485007, 0.0120141 , 0.04826106,
        0.02095791, 0.00054798, 0.03176954, 0.00470337, 0.06457451,
        0.06969912, 0.05212089, 0.02255993, 0.07352586, 0.01822554,
        0.04413363, 0.07068298, 0.02451418, 0.0139926 , 0.0334258 ,
        0.04975198, 0.06431437, 0.02071189, 0.03112417, 0.02834546]),
 array([0.07077969, 0.03185596, 0.02611819, 0.07457894, 0.04195951,
        0.02571833, 0.00556523, 0.02774478, 0.01693018, 0.02187198,
        0.00283476, 0.03150771, 0.00275684, 0.01506104, 0.0768279 ,
        0.03898755, 0.05910078, 0.0202408 , 0.07297405, 0.0051162 ,
        0.01016559, 0.03414657, 0.06632396, 0.00340036, 0.02504692,
        0.03058891, 0.03377652, 0.06996117, 0.02779279, 0.03026679])]

In [62]:
df_GRM = (good_weights_list[0] * df_monthly).sum(axis=1)
df_GRM[df_GRM.index.isin(df_MXWO_topdd.index)]

IndexError: list index out of range

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
plt.plot((1+df_GRM).cumprod())

In [ ]:
plt.plot((1+df_MXWO_return).cumprod().squeeze())

In [ ]:
plt.plot((1 + (0.5 * df_GRM) + (0.5 * df_MXWO_return.squeeze())).cumprod())

In [ ]:
# worst 5% dd MSCI month?
# limit dd by at 25% (GRM +25% of )
# monthly data? 
# plus correlation + contribution to risk 
# rebalance monthly 
# show both one off and rolling 3yrs

In [ ]:
# drawdown as a constraint, 
# multiple objects? 
# just do monte carlo with monthly data rather than optimiser (with 20bps...)

In [108]:
worst_MXWO_months

DatetimeIndex(['2008-10-31', '2020-03-31', '2008-09-30', '2009-02-28',
               '2010-05-31', '2012-05-31', '2011-09-30', '2009-01-31'],
              dtype='datetime64[ns]', name='Date', freq=None)